In [1]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.device_count())
if torch.cuda.is_available():
    print(torch.cuda.get_device_name(0))

True
1
NVIDIA GeForce RTX 3080 Laptop GPU


In [4]:
from dotenv import load_dotenv
import os
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_huggingface import HuggingFacePipeline

# Cargar variables de entorno
load_dotenv()
hf_token = os.getenv("HUGGINGFACE_TOKEN")

# Cargar el tokenizer y el modelo manualmente
tokenizer = AutoTokenizer.from_pretrained(
    "meta-llama/Llama-3.2-3B-Instruct",
    token=hf_token
)
# Si no tiene pad_token, lo asignamos al eos_token
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-3.2-3B-Instruct",
    token=hf_token
)

Loading checkpoint shards: 100%|██████████| 2/2 [00:04<00:00,  2.05s/it]


In [5]:
# Crear el pipeline de HuggingFace
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=100,
    device=0
)

# Integrar con LangChain
llm = HuggingFacePipeline(pipeline=pipe)

Device set to use cuda:0


In [6]:
llm.invoke("What is Hugging Face?")

'What is Hugging Face? Hugging Face is an open-source software library for natural language processing (NLP) tasks. It was created by Hugging Face LLC and is widely used in the field of NLP. The library provides a wide range of pre-trained models and tools for tasks such as text classification, sentiment analysis, and language translation.\nHugging Face provides a simple and intuitive API for accessing and using these models, making it easy for developers to integrate NLP capabilities into their applications. The library also includes a range'

# Nodo inicial 

In [54]:
from typing import TypedDict
from langgraph.graph import StateGraph, END
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# Paso 1: Define el esquema de estado
class AgentState(TypedDict):
    input: str
    mode: str

# Paso 2: Define el prompt para clasificar el modo
prompt = PromptTemplate(
    input_variables=["input"],
    template=(
        "Clasifica el siguiente mensaje del usuario como 'modo libre' o 'modo guiado'. "
        "Solo responde con una de esas dos opciones.\n\n"
        "Mensaje: {input}\n"
        "Clasificación:"
    )
)

# Paso 3: Crea la cadena de clasificación
classifier_chain = LLMChain(
    llm=llm,  # Tu LLM de HuggingFacePipeline
    prompt=prompt
)

# Paso 4: Define la función de clasificación (corregida)
def classify_mode(state):
    user_input = state["input"]
    result = classifier_chain.run(input=user_input)
    # Buscar la línea que contiene 'Clasificación:'
    mode = "modo guiado"  # Valor por defecto
    for line in result.splitlines():
        if "clasificación:" in line.lower():
            after_colon = line.split(":", 1)[-1].strip().lower()
            if "libre" in after_colon:
                mode = "modo libre"
            else:
                mode = "modo guiado"
            break
    state["mode"] = mode
    return state

# Paso 5: Crea el grafo de LangGraph
graph = StateGraph(state_schema=AgentState)
graph.add_node("clasificar", classify_mode)
graph.set_entry_point("clasificar")
graph.add_edge("clasificar", END)

# Paso 6: Compila el grafo
app = graph.compile()

# Paso 7: Usa el agente con input desde teclado
while True:
    user_input = input("Escribe tu mensaje (o 'salir' para terminar): ")
    if user_input.lower() == "salir":
        break
    result = app.invoke({"input": user_input})
    print("Modo detectado:", result["mode"])

Modo detectado: modo guiado
Modo detectado: modo guiado
Modo detectado: modo libre
Modo detectado: modo libre
Modo detectado: modo libre
Modo detectado: modo guiado
Modo detectado: modo libre
Modo detectado: modo guiado


In [1]:
from typing import TypedDict
from langgraph.graph import StateGraph, END
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import os

# Cargar variables de entorno
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

class AgentState(TypedDict):
    input: str
    mode: str

prompt = PromptTemplate(
    input_variables=["input"],
    template=(
        "Clasifica el siguiente mensaje del usuario como 'modo libre' o 'modo guiado'. "
        "Solo responde con una de esas dos opciones.\n\n"
        "Mensaje: {input}\n"
        "Clasificación:"
    )
)

llm = ChatOpenAI(
    model="gpt-4o",
    temperature=1,
    openai_api_key=openai_api_key
)

classifier_chain = LLMChain(
    llm=llm,
    prompt=prompt
)

C:\Users\jgome\AppData\Local\Temp\ipykernel_2468\967007799.py:33: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  classifier_chain = LLMChain(


In [2]:
respuesta = classifier_chain.invoke(input="quiero solo preguntar")

In [3]:
respuesta

{'input': 'quiero solo preguntar', 'text': 'modo libre'}

In [4]:
respuesta = classifier_chain.invoke(input="quiero ir paso a paso")
respuesta

{'input': 'quiero ir paso a paso', 'text': 'modo guiado'}

In [5]:
def classify_mode(state):
    user_input = state["input"]
    result = classifier_chain.run(input=user_input)
    mode = "modo guiado"
    for line in result.splitlines():
        if "clasificación:" in line.lower():
            after_colon = line.split(":", 1)[-1].strip().lower()
            if "libre" in after_colon:
                mode = "modo libre"
            else:
                mode = "modo guiado"
            break
    state["mode"] = mode
    return state

graph = StateGraph(state_schema=AgentState)
graph.add_node("clasificar", classify_mode)
graph.set_entry_point("clasificar")
graph.add_edge("clasificar", END)

app = graph.compile()

while True:
    user_input = input("Escribe tu mensaje (o 'salir' para terminar): ")
    if user_input.lower() == "salir":
        break
    result = app.invoke({"input": user_input})
    print("Modo detectado:", result["mode"])

C:\Users\jgome\AppData\Local\Temp\ipykernel_25344\4117726483.py:3: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = classifier_chain.run(input=user_input)


Modo detectado: modo guiado
Modo detectado: modo guiado
Modo detectado: modo guiado
Modo detectado: modo guiado


In [6]:
from typing import TypedDict
from langgraph.graph import StateGraph, END
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import os

# Cargar variables de entorno
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

# Esquema de estado
class AgentState(TypedDict):
    input: str
    mode: str
    output: str

# Prompt para clasificar el modo
prompt = PromptTemplate(
    input_variables=["input"],
    template=(
        "Clasifica el siguiente mensaje del usuario como 'modo libre' o 'modo guiado'. "
        "Solo responde con una de esas dos opciones.\n\n"
        "Mensaje: {input}\n"
        "Clasificación:"
    )
)

# LLM de OpenAI
llm = ChatOpenAI(
    model="o4-mini-2025-04-16",
    openai_api_key=openai_api_key
)

# Cadena de clasificación
classifier_chain = LLMChain(
    llm=llm,
    prompt=prompt
)

# Nodo de clasificación
def classify_mode(state):
    user_input = state["input"]
    respuesta = classifier_chain.invoke(input=user_input)
    modo = respuesta.get("text", "").strip().lower()
    if "libre" in modo:
        state["mode"] = "modo libre"
    else:salir
        state["mode"] = "modo guiado"
    return state

# Nodo para modo guiado
def modo_guiado(state):
    # Aquí puedes poner el flujo guiado real, por ahora solo un mensaje
    state["output"] = "Iniciando modo guiado: te explicaré paso a paso."
    return state

# Nodo para modo libre
def modo_libre(state):
    # Aquí puedes poner el flujo libre real, por ahora solo un mensaje
    state["output"] = "Iniciando modo libre: puedes preguntar lo que quieras."
    return state

# Construcción del grafo
graph = StateGraph(state_schema=AgentState)
graph.add_node("clasificar", classify_mode)
graph.add_node("guiado", modo_guiado)
graph.add_node("libre", modo_libre)

# Flujo: clasificar -> guiado/libre -> END
graph.set_entry_point("clasificar")
graph.add_conditional_edges(
    "clasificar",
    lambda state: state["mode"],
    {
        "modo guiado": "guiado",
        "modo libre": "libre"
    }
)
graph.add_edge("guiado", END)
graph.add_edge("libre", END)

# Compilar el grafo
app = graph.compile()

# Loop de interacción
while True:
    user_input = input("Escribe tu mensaje (o 'salir' para terminar): ")
    if user_input.lower() == "salir":
        break
    result = app.invoke({"input": user_input, "mode": "", "output": ""})
    print("Modo detectado:", result["mode"])
    print(result["output"])

Modo detectado: modo libre
Iniciando modo libre: puedes preguntar lo que quieras.
Modo detectado: modo libre
Iniciando modo libre: puedes preguntar lo que quieras.
Modo detectado: modo guiado
Iniciando modo guiado: te explicaré paso a paso.
Modo detectado: modo guiado
Iniciando modo guiado: te explicaré paso a paso.
Modo detectado: modo guiado
Iniciando modo guiado: te explicaré paso a paso.


## Nodo Quiz

In [5]:
import random
import openai
import json
from langchain.prompts import ChatPromptTemplate

# 1. Clase Estado simple
class Estado:
    def __init__(self):
        self.nivel = None
        self.fortalezas = []
        self.debilidades = []

# 2. Preguntas por nivel y tema
quiz_preguntas = {
    "basico": [
        {"tema": "Variables aleatorias", "pregunta": "¿Qué es una variable aleatoria?"},
        {"tema": "Probabilidad", "pregunta": "¿Qué es la probabilidad clásica?"},
        {"tema": "Distribuciones simples", "pregunta": "¿Qué es una distribución uniforme?"},
        {"tema": "Eventos", "pregunta": "¿Qué es un evento en probabilidad?"}
    ],
    "intermedio": [
        {"tema": "Desviación estándar", "pregunta": "¿Qué es la desviación estándar?"},
        {"tema": "Medidas de dispersión", "pregunta": "¿Qué es la varianza?"},
        {"tema": "Estadística descriptiva", "pregunta": "¿Qué es la media aritmética?"},
        {"tema": "Distribuciones", "pregunta": "¿Qué es una distribución normal?"}
    ],
    "avanzado": [
        {"tema": "Probabilidad conjunta", "pregunta": "¿Cómo se calcula la probabilidad conjunta de dos eventos independientes?"},
        {"tema": "Teorema de Bayes", "pregunta": "Explica el teorema de Bayes con un ejemplo."},
        {"tema": "Distribuciones avanzadas", "pregunta": "¿Qué es una distribución binomial?"},
        {"tema": "Inferencia", "pregunta": "¿Qué es una estimación puntual en inferencia estadística?"}
    ]
}

# 3. Template del prompt
prompt_template = ChatPromptTemplate.from_messages([
    ("system",
     """Eres un experto en educación. Evalúa las siguientes respuestas del usuario a preguntas de probabilidad y estadística.
Para cada respuesta, califica de 0 a 5 (donde 0 es incorrecta y 5 es perfecta), explica brevemente la calificación.
Al final, resume las fortalezas y debilidades del usuario por tema y sugiere el nivel adecuado (básico, intermedio, avanzado) según el promedio de los puntajes:
- Básico: promedio < 2.5
- Intermedio: 2.5 <= promedio < 4
- Avanzado: promedio >= 4

Devuelve la respuesta SOLO en formato JSON con la siguiente estructura:
{{
  "nivel": "basico/intermedio/avanzado",
  "fortalezas": ["tema1", "tema2", ...],
  "debilidades": ["tema1", "tema2", ...],
  "detalle": [
    {{
      "pregunta": "...",
      "respuesta": "...",
      "tema": "...",
      "puntaje": 0-5,
      "feedback": "..."
    }},
    ...
  ]
}}

Respuestas del usuario:
{respuestas_usuario}
""")
])

# 4. Nodo de quiz de nivel
def nodo_quiz_nivel(state):
    print("\nVamos a hacer un quiz para conocer tu nivel.")
    respuestas_usuario = []
    for nivel, preguntas in quiz_preguntas.items():
        seleccionadas = random.sample(preguntas, 1)
        for q in seleccionadas:
            resp = input(q["pregunta"] + " (responde brevemente): ")
            respuestas_usuario.append({
                "nivel": nivel,
                "tema": q["tema"],
                "pregunta": q["pregunta"],
                "respuesta": resp
            })
    # Prepara el prompt usando el template
    prompt = prompt_template.format(respuestas_usuario=str(respuestas_usuario))
    # Llama al LLM
    response = openai.chat.completions.create(
        model="o4-mini-2025-04-16",
        messages=[
            {"role": "system", "content": prompt}
        ]
    )
    diagnostico = response.choices[0].message.content
    print("\n--- Diagnóstico del agente ---")
    print(diagnostico)
    resultado = json.loads(diagnostico)
    state.nivel = resultado["nivel"]
    state.fortalezas = resultado["fortalezas"]
    state.debilidades = resultado["debilidades"]
    state.detalle = resultado["detalle"]
    print(f"\nNivel detectado: {state.nivel}")
    print(f"Fortalezas: {state.fortalezas}")
    print(f"Debilidades: {state.debilidades}")
    
    return state

# 5. Prueba el nodo
if __name__ == "__main__":
    estado = Estado()
    nodo_quiz_nivel(estado)
    print("\nResumen final:")
    print("Nivel:", estado.nivel)
    print("Fortalezas:", estado.fortalezas)
    print("Debilidades:", estado.debilidades)


Vamos a hacer un quiz para conocer tu nivel.

--- Diagnóstico del agente ---
{"nivel":"basico","fortalezas":[],"debilidades":["Variables aleatorias","Estadística descriptiva","Teorema de Bayes"],"detalle":[{"pregunta":"¿Qué es una variable aleatoria?","respuesta":"c","tema":"Variables aleatorias","puntaje":0,"feedback":"No se proporciona ninguna definición ni explicación; la respuesta es inválida."},{"pregunta":"¿Qué es la media aritmética?","respuesta":"c","tema":"Estadística descriptiva","puntaje":0,"feedback":"No se responde la pregunta, solo se indica una letra sin contenido conceptual."},{"pregunta":"Explica el teorema de Bayes con un ejemplo.","respuesta":"c","tema":"Teorema de Bayes","puntaje":0,"feedback":"No hay explicación ni ejemplo; la respuesta es completamente incompleta."}]}

Nivel detectado: basico
Fortalezas: []
Debilidades: ['Variables aleatorias', 'Estadística descriptiva', 'Teorema de Bayes']

Resumen final:
Nivel: basico
Fortalezas: []
Debilidades: ['Variables ale

In [7]:
import random
import json
from typing import List
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain
from langchain_core.output_parsers.json import JsonOutputParser

# 1. Clase Estado simple
class Estado:
    def __init__(self):
        self.nivel = None
        self.fortalezas = []
        self.debilidades = []
        self.detalle = []

# 2. Preguntas por nivel y tema
quiz_preguntas = {
    "basico": [
        {"tema": "Variables aleatorias", "pregunta": "¿Qué es una variable aleatoria?"},
        {"tema": "Probabilidad", "pregunta": "¿Qué es la probabilidad clásica?"},
        {"tema": "Distribuciones simples", "pregunta": "¿Qué es una distribución uniforme?"},
        {"tema": "Eventos", "pregunta": "¿Qué es un evento en probabilidad?"}
    ],
    "intermedio": [
        {"tema": "Desviación estándar", "pregunta": "¿Qué es la desviación estándar?"},
        {"tema": "Medidas de dispersión", "pregunta": "¿Qué es la varianza?"},
        {"tema": "Estadística descriptiva", "pregunta": "¿Qué es la media aritmética?"},
        {"tema": "Distribuciones", "pregunta": "¿Qué es una distribución normal?"}
    ],
    "avanzado": [
        {"tema": "Probabilidad conjunta", "pregunta": "¿Cómo se calcula la probabilidad conjunta de dos eventos independientes?"},
        {"tema": "Teorema de Bayes", "pregunta": "Explica el teorema de Bayes con un ejemplo."},
        {"tema": "Distribuciones avanzadas", "pregunta": "¿Qué es una distribución binomial?"},
        {"tema": "Inferencia", "pregunta": "¿Qué es una estimación puntual en inferencia estadística?"}
    ]
}

# 3. Template del prompt
prompt_template = ChatPromptTemplate.from_messages([
    ("system",
     """Eres un experto en educación. Evalúa las siguientes respuestas del usuario a preguntas de probabilidad y estadística.
Para cada respuesta, califica de 0 a 5 (donde 0 es incorrecta y 5 es perfecta), explica brevemente la calificación.
Al final, resume las fortalezas y debilidades del usuario por tema y sugiere el nivel adecuado (básico, intermedio, avanzado) según el promedio de los puntajes:
- Básico: promedio < 2.5
- Intermedio: 2.5 <= promedio < 4
- Avanzado: promedio >= 4

Devuelve la respuesta SOLO en formato JSON con la siguiente estructura:
{{
  "nivel": "basico/intermedio/avanzado",
  "fortalezas": ["tema1", "tema2", ...],
  "debilidades": ["tema1", "tema2", ...],
  "detalle": [
    {{
      "pregunta": "...",
      "respuesta": "...",
      "tema": "...",
      "puntaje": 0-5,
      "feedback": "..."
    }},
    ...
  ]
}}

Respuestas del usuario:
{respuestas_usuario}
""")
])

# 4. Nodo de quiz de nivel usando LLMChain y JsonOutputParser
def nodo_quiz_nivel(state):
    print("\nVamos a hacer un quiz para conocer tu nivel.")
    respuestas_usuario = []
    for nivel, preguntas in quiz_preguntas.items():
        seleccionadas = random.sample(preguntas, 1)  # 1 aleatoria por nivel
        for q in seleccionadas:
            resp = input(q["pregunta"] + " (responde brevemente): ")
            respuestas_usuario.append({
                "nivel": nivel,
                "tema": q["tema"],
                "pregunta": q["pregunta"],
                "respuesta": resp
            })

    # 5. Prepara el LLM y la chain
    llm = ChatOpenAI(model="o4-mini-2025-04-16", temperature=1)
    parser = JsonOutputParser()
    chain = LLMChain(
        llm=llm,
        prompt=prompt_template,
        output_parser=parser
    )

    # 6. Ejecuta la chain
    result = chain.invoke({"respuestas_usuario": str(respuestas_usuario)})

    # 7. Imprime el resultado crudo para depuración
    print("\n--- Resultado crudo del LLM ---")
    print(result)

    # 8. Si el resultado es un dict y tiene 'text', extrae los campos desde ahí
    if isinstance(result, dict) and "text" in result:
        data = result["text"]
        state.nivel = data["nivel"]
        state.fortalezas = data["fortalezas"]
        state.debilidades = data["debilidades"]
        state.detalle = data["detalle"]
        print(f"\nNivel detectado: {state.nivel}")
        print(f"Fortalezas: {state.fortalezas}")
        print(f"Debilidades: {state.debilidades}")
    else:
        print("\nNo se pudo extraer el JSON esperado. Revisa el resultado arriba.")

    return state

# 5. Prueba el nodo
if __name__ == "__main__":
    estado = Estado()
    nodo_quiz_nivel(estado)
    print("\nResumen final:")
    print("Nivel:", estado.nivel)
    print("Fortalezas:", estado.fortalezas)
    print("Debilidades:", estado.debilidades)


Vamos a hacer un quiz para conocer tu nivel.

--- Resultado crudo del LLM ---
{'respuestas_usuario': "[{'nivel': 'basico', 'tema': 'Distribuciones simples', 'pregunta': '¿Qué es una distribución uniforme?', 'respuesta': 'una distribucion donde la probabilidad es igual'}, {'nivel': 'intermedio', 'tema': 'Medidas de dispersión', 'pregunta': '¿Qué es la varianza?', 'respuesta': 'que tan alejados estan los datos del promedio'}, {'nivel': 'avanzado', 'tema': 'Distribuciones avanzadas', 'pregunta': '¿Qué es una distribución binomial?', 'respuesta': 'una distribucion donde los suceson solo pueden ser exito o fracaso'}]", 'text': {'nivel': 'intermedio', 'fortalezas': ['Distribuciones simples', 'Medidas de dispersión'], 'debilidades': ['Distribuciones avanzadas'], 'detalle': [{'pregunta': '¿Qué es una distribución uniforme?', 'respuesta': 'una distribucion donde la probabilidad es igual', 'tema': 'Distribuciones simples', 'puntaje': 4, 'feedback': 'Correcto concepto básico, pero faltó menciona

# Nodo plan de estudio 

In [11]:
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI  # O el import de tu LLM preferido

# Simulación de estado con debilidades
class Estado:
    def __init__(self, nivel, debilidades):
        self.nivel = nivel
        self.debilidades = debilidades
        self.temas = []
        self.tema_actual = 0

# Prompt template para el plan de estudio
prompt = ChatPromptTemplate.from_template(
    """
Eres un tutor experto en estadística y probabilidad.
El estudiante tiene el nivel: {nivel}.
Sus debilidades principales son: {debilidades}.
Crea un plan de estudio personalizado de exactamente 3 temas grandes, enfocados en esas debilidades.
Enumera los temas de forma clara y breve.
"""
)

# Instancia del LLM (ajusta el modelo según tu entorno)
llm = ChatOpenAI(model="o4-mini", temperature=1)

# Chain para generar el plan
chain = LLMChain(llm=llm, prompt=prompt)

def nodo_plan_estudio(state: Estado) -> Estado:
    # Llama al LLM para obtener el plan personalizado
    debilidades_str = ", ".join(state.debilidades)
    respuesta = chain.run(nivel=state.nivel, debilidades=debilidades_str)
    print("\nPlan de estudio personalizado:")
    print(respuesta)
    # Extrae los temas del texto generado (puedes mejorar esto según el formato de respuesta)
    temas = [line.strip("- ").strip() for line in respuesta.split("\n") if line.strip()]
    state.temas = temas[:3]  # Solo los 3 primeros temas
    state.tema_actual = 0
    return state

# Prueba aislada
estado_prueba = Estado(nivel="intermedio", debilidades=["probabilidad condicional", "distribuciones", "teorema de Bayes","medidad de tendencia central", "regresiones"])
nodo_plan_estudio(estado_prueba)


Plan de estudio personalizado:
Plan de estudio (3 temas grandes)

1. Probabilidad condicional y Teorema de Bayes  
   • Definición de probabilidad condicional  
   • Regla de la cadena y teorema de la probabilidad total  
   • Enunciado y demostración de Bayes  
   • Ejercicios con árboles y tablas de contingencia  

2. Distribuciones y medidas de tendencia central  
   • Distribuciones discretas (Binomial, Poisson) y continuas (Normal, Exponencial)  
   • Función de densidad/masa, media y varianza  
   • Cálculo e interpretación de media, mediana y moda  
   • Relación entre forma de la distribución y medidas de tendencia  

3. Regresión lineal y análisis de relaciones  
   • Modelo de regresión simple: estimación de coeficientes por MCO  
   • Interpretación de pendiente, ordenada al origen y coeficiente de determinación (R²)  
   • Supuestos básicos (linealidad, homocedasticidad, normalidad de errores)  
   • Validación del modelo y predicción  



In [6]:
prompt_plan = ChatPromptTemplate.from_template(
    """
Eres un tutor experto en estadística y probabilidad.
El estudiante tiene el nivel: {nivel}.
Sus debilidades principales son: {debilidades}.

Crea un plan de estudio personalizado siguiendo EXACTAMENTE este formato:

Plan de estudio (3 temas grandes)

1. [Nombre del Tema 1]
   • [Subtema 1.1]
   • [Subtema 1.2]
   • [Subtema 1.3]
   • [Subtema 1.4]

2. [Nombre del Tema 2]
   • [Subtema 2.1]
   • [Subtema 2.2]
   • [Subtema 2.3]
   • [Subtema 2.4]

3. [Nombre del Tema 3]
   • [Subtema 3.1]
   • [Subtema 3.2]
   • [Subtema 3.3]
   • [Subtema 3.4]

Los temas deben enfocarse en las debilidades mencionadas.
Cada tema debe tener exactamente 4 subtemas.
"""
)

def parse_plan(texto):
    """Función para parsear la respuesta y extraer temas y subtemas"""
    temas = {}
    tema_actual = None

    for linea in texto.split('\n'):
        linea = linea.strip()
        if linea.startswith(('1.', '2.', '3.')):
            tema_actual = linea[3:].strip()
            temas[tema_actual] = []
        elif linea.startswith('•'):
            if tema_actual:
                temas[tema_actual].append(linea[2:].strip())

    return temas

def nodo_plan_estudio(state: Estado) -> Estado:
    debilidades_str = ", ".join(state.debilidades)
    chain = prompt_plan | llm
    respuesta = chain.run(nivel=state.nivel, debilidades=debilidades_str)

    # Parsear la respuesta
    temas_subtemas = parse_plan(respuesta)

    # Actualizar el estado
    state.temas = list(temas_subtemas.keys())
    state.subtemas = temas_subtemas
    state.tema_actual = 0

    print("\nPlan de estudio personalizado:")
    print(respuesta)
    return state

In [7]:
tema_ejemplo = "Probabilidad condicional y Teorema de Bayes"
subtemas_ejemplo = [
    "Definición de probabilidad condicional",
    "Regla de la cadena y teorema de la probabilidad total",
    "Enunciado y demostración de Bayes",
    "Ejercicios con árboles y tablas de contingencia"
]

In [ ]:
parseplan = parse_plan(